### Before Running the Program - Preparation Part 1
#### AVA Dataset Preparation
#### Before run the .ipynb file, please follow the link and download AVA image database first.
#### Link to download AVA: http://academictorrents.com/details/71631f83b11d3d79d8f84efe0a7e12f0ac001460
#### Note: Need a Bittorrent client to download all the files
#### After finishing downloading, we have an folder called "AVA_dataset" which contains all the 7z files of images
#### Unzip all the files to the folder "AVA_dataset"
#### Move the AVA_dataset which contains all AVA images to the folder "database" contained in our project

### Before Running the Program - Preparation Part 2
#### Download the VOC test set from PASCAL VOC Challenge performance evaluation and download server
#### Link to download: http://host.robots.ox.ac.uk:8080/
#### Note: You will need to register if it is your first time to download the test dataset
#### After login with your username and password, click the link of "PASCAL Visual Object Classes Challenge 2012 (VOC2012)" to download the database. 

## Program Start ...

In [2]:
import os
import tarfile
import shutil
import glob
import requests
import zipfile
import io
import urllib
import cv2
import pandas as pd
import sys
import time
from tqdm import tqdm
from clint.textui import progress
from sys import stdout

In [3]:
if sys.version_info[0] < 3:
    raise Exception("Must be using Python 3")

In [4]:
from urllib.request import urlretrieve

In [5]:
def downloader(url,parameters, file_name):
    session = requests.Session()
    response = session.get(url,params=parameters, stream=True)
    if parameters is not None:
        token = None
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                token = value
        if token:
            params = {'id': '0B7sjGeF4f3FYQUVlZ3ZOai1ieEU', 'confirm': token}
            response = session.get('https://docs.google.com/uc?export=download', params=params, stream=True)
    with open(file_name, 'wb') as f:
        total_size = 0
        if file_name != 'emotic.zip':
            total_size = int(response.headers.get('content-length'))
        else:
            total_size = 3602935747
        with tqdm(total = int(total_size/32768), position=0, leave=True) as pbar:
            for chunk in response.iter_content(32768):
                f.write(chunk)
                pbar.update()
            pbar.close()
    print("%s downloading - Done!" %file_name)

### Download VOC Train Validation Set...

In [5]:
url = 'http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar'
parameters = None
file_name = 'voc.tar'
downloader(url, parameters, file_name)

61025it [02:57, 344.43it/s]                           


voc.tar downloading - Done!


### Extract VOC Train Validation tar File, Rename Images, and Move Image Files

In [16]:
tar = tarfile.open('voc.tar')
tar.extractall('database/voc_emotic_ava')
files = "database/voc_emotic_ava/VOCdevkit/VOC2012/JPEGImages/*"
with tqdm(total = int(len(glob.glob(files))), position=0, leave=True) as pbar:
    for path in glob.glob(files):
        name = path.split("/")[-1]
        newp1 = path.replace(name,"")+"JPEGImages__"+name
        newp2 = path.replace(name, "")+"VOC2012__"+name
        os.rename(path, newp1)
        shutil.copy(newp1, 'database/voc_emotic_ava/')
        os.rename(newp1, newp2)
        shutil.copy(newp2, 'database/voc_emotic_ava/')
        os.remove(newp2)
        pbar.update()
    pbar.update()
    pbar.close()
shutil.rmtree('database/voc_emotic_ava/VOCdevkit')
print("VOC - Done!")

17126it [00:31, 538.74it/s]                           


VOC - Done!


### Extract VOC Test tar File, Rename Images, and Move Image Files

In [17]:
tar = tarfile.open('voc_test.tar')
tar.extractall('database/voc_emotic_ava')
files = "database/voc_emotic_ava/VOCdevkit/VOC2012/JPEGImages/*"
with tqdm(total = int(len(glob.glob(files))), position=0, leave=True) as pbar:
    for path in glob.glob(files):
        name = path.split("/")[-1]
        newp1 = path.replace(name,"")+"JPEGImages__"+name
        newp2 = path.replace(name, "")+"VOC2012__"+name
        os.rename(path, newp1)
        shutil.copy(newp1, 'database/voc_emotic_ava/')
        os.rename(newp1, newp2)
        shutil.copy(newp2, 'database/voc_emotic_ava/')
        os.remove(newp2)
        pbar.update()
    pbar.update()
    pbar.close()
shutil.rmtree('database/voc_emotic_ava/VOCdevkit')
print("VOC_test_set_2012 - Done!")

16136it [00:23, 686.37it/s]                            


VOC_test_set_2012 - Done!


### Download Emotic...

In [7]:
url = 'https://docs.google.com/uc?export=download'
parameters = {'id': '0B7sjGeF4f3FYQUVlZ3ZOai1ieEU'}
file_name = 'emotic.zip'
downloader(url, parameters, file_name)

109956it [01:12, 1518.22it/s]                            


emotic.zip downloading - Done!


### Extract Emotic Zip File, Rename Images, and Move Image Files

In [8]:
zipfile.ZipFile('emotic.zip', 'r').extractall('database/voc_emotic_ava')
folders = os.listdir("database/voc_emotic_ava/emotic/")
with tqdm(total = len(folders), position=1, leave=True) as pbar:
    for folder in folders: 
        files = "database/voc_emotic_ava/emotic/"+folder + "/images/*"
        for path in glob.glob(files):
            name = path.split("/")[-1]
            newp = path.replace(name,"")+"EMOTIC__"+name
            os.rename(path, newp)
            shutil.copy(newp, 'database/voc_emotic_ava/')
            os.remove(newp)
        pbar.update()
    pbar.close()
shutil.rmtree('database/voc_emotic_ava/emotic')
shutil.rmtree('database/voc_emotic_ava/__MACOSX')
print("Emotic - Done!")


100%|██████████| 5/5 [00:08<00:00,  1.65s/it]

Emotic - Done!


### Download Blur Detection Dataset ...

In [14]:
url = 'http://www.cse.cuhk.edu.hk/~leojia/projects/dblurdetect/data/BlurDatasetImage.zip'
parameters = None
file_name = 'blur.zip'
downloader(url, parameters, file_name)

3731it [00:17, 208.82it/s]                          

blur.zip downloading - Done!


### Extract Emotic Zip File, Rename Images, and Move Image Files

In [15]:
zipfile.ZipFile('blur.zip','r').extractall('database/blur_dataset')
for path in glob.glob('database/blur_dataset/image/*'):
    shutil.copy(path, 'database/blur_dataset/')
    os.remove(path)
shutil.rmtree('database/blur_dataset/image')
print("Blur dataset - Done!")

Blur dataset - Done!


### Rename and Move AVA Images

In [11]:
if os.path.isdir('database/AVA_dataset') == True:
    files = 'database/AVA_dataset/images/*'
    for path in tqdm(glob.glob(files)):
        name = path.split("/")[-1]
        newp = path.replace(name, "")+"AVA__"+name
        os.rename(path, newp)
        shutil.move(newp, 'database/voc_emotic_ava/')
    shutil.rmtree('database/voc_emotic_ava/AVA_dataset')
    print("AVA - Done!")

### Clean Image Not Been Used ...

In [11]:
df_ims = pd.read_csv('labels_image.csv')
ims = df_ims['name']
ims = ims.values.tolist()

In [12]:
folders = os.listdir('database')

In [13]:
for folder in folders:
    files = 'database/'+folder+"/*"
    for path in glob.glob(files):
        name = path.replace("database/","")
        if name not in ims:
            os.remove(path)
print("Cleaning - Done!")

Cleaning - Done!


### Sampling Patches from Images

In [76]:
try:
    if os.path.isdir('database/patches') == False:
        os.mkdir('database/patches')
except OSError:
    print ("Creation of the directory %s failed" % 'patch')

In [77]:
def patch_sampling(im_path, patch_index, x, y, width, height):
    img=cv2.imread(im_path)
    patch = img[y:y+height, x:x+width]
    name = im_path.split('/')[-1]
    p_path = 'database/patches/'+name+"_"+"patch_"+str(patch_index)+".jpg"
    cv2.imwrite(p_path,patch)

In [79]:
df_coor = pd.read_csv('all_patches.csv')

In [82]:
for folder in folders:
    if folder != 'patches':
        im_paths = 'database/'+folder+"/*"
        im_name = im_path.split('/')[-1]
        patch_name = "patches/"+im_name
        row = df_coor.loc[df_coor['name_patch']==patch_name]
        if len(row) == 1:
            x1 = int(row['top_patch_1'].values.tolist()[0])
            y1 = int(row['left_patch_1'].values.tolist()[0])
            x2 = int(row['top_patch_2'].values.tolist()[0])
            y2 = int(row['left_patch_2'].values.tolist()[0])
            x3 = int(row['top_patch_3'].values.tolist()[0])
            y3 = int(row['left_patch_3'].values.tolist()[0])
            h1 = int(row['height_patch_1'].values.tolist()[0])
            w1 = int(row['width_patch_1'].values.tolist()[0])
            h2 = int(row['height_patch_2'].values.tolist()[0])
            w2 = int(row['width_patch_2'].values.tolist()[0])
            h3 = int(row['height_patch_3'].values.tolist()[0])
            w3 = int(row['width_patch_3'].values.tolist()[0])
            patch_sampling(im_path, 1, x1, y1, w1, h1)
            patch_sampling(im_path, 2, x2, y2, w2, h2)
            patch_sampling(im_path, 3, x3, y3, w3, h3)
print("Patch sampling - Done!")

Patch sampling - Done!


### Congratulations! It's All Set! Now all you need is in the database folder. :)